In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# Step 1: Load Models
yolo_face_model = YOLO("../models/yolov11n-face.pt").to('cpu')  # YOLO Face detection
classification_model = YOLO("../student_faces/yolo_classification/weights/best.pt").to('cpu')  # YOLO Classification

# Define class labels for YOLO classification
classes = ['245322748034', '245322748042', '245322748045']

# Step 2: Main Pipeline Function
def process_image(image_path):
    """
    Pipeline: 
    - Detect faces using YOLO Face Detector
    - Classify each detected face using YOLO Classification Model
    - Display the class predictions with confidence scores on the image
    """
    # Load and prepare the image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 1. Detect Faces using YOLO Face Detector
    print("Running YOLO Face Detection...")
    face_results = yolo_face_model.predict(source=image_path, imgsz=224, conf=0.5)
    face_boxes = face_results[0].boxes.xyxy.cpu().numpy()  # Bounding box predictions

    if len(face_boxes) == 0:
        print("No faces detected in the image.")
        return

    for i, box in enumerate(face_boxes):
        x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates
        cropped_face = img_rgb[y1:y2, x1:x2]  # Crop the detected face

        if cropped_face.size == 0:
            print(f"Invalid face region for box {i}.")
            continue

        # Resize face to 224x224 for YOLO classification
        face_resized = cv2.resize(cropped_face, (224, 224))

        # 2. Predict Class using YOLO Classification Model
        print("Running YOLO Classification Model...")
        classification_results = classification_model.predict(source=face_resized, imgsz=224, conf=0.5)

        if hasattr(classification_results[0], 'probs') and classification_results[0].probs is not None:
            class_probs = classification_results[0].probs.data  # Class probabilities
            predicted_class_idx = class_probs.argmax()  # Index of the most confident class
            predicted_class = classes[predicted_class_idx]  # Map to class labels
            confidence_classification = class_probs[predicted_class_idx].item()

            print(f"Face {i+1} - Class: {predicted_class}, Confidence: {confidence_classification:.2f}")

            # If confidence > 90%, draw the bounding box and label
            if confidence_classification > 0.9:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                label = f"{predicted_class} ({confidence_classification * 100:.1f}%)"
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        else:
            print(f"Face {i+1} - No class predictions returned by YOLO.")
            continue

    # Display the final image with bounding boxes
    cv2.imshow("Detected Faces", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Step 3: Run Pipeline on a Test Image
image_path = "./students/245322748034/Snapchat-1329597328.jpg"  # Replace with your test image path
process_image(image_path)


Running YOLO Face Detection...

image 1/1 c:\Users\vishw\Desktop\mini project\automated-attendance\students\245322748034\Snapchat-1329597328.jpg: 224x128 1 face, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 128)
Running YOLO Classification Model...

0: 224x224 245322748034 0.98, 245322748045 0.02, 245322748042 0.00, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Face 1 - Class: 245322748034, Confidence: 0.98
